In [ ]:
import dotenv from "dotenv";
dotenv.config({ path: '.env.development.local' });
// Deno imports
import { Stagehand } from "@browserbasehq/stagehand";
import { z } from 'zod';

const cdpPort = 9222;
let stagehand = null;



In [ ]:
//Initialize Stagehand
(async () => {
  stagehand = new Stagehand({
    verbose: 1, // Verbosity level for logging: 0 = silent, 1 = info, 2 = all
    domSettleTimeoutMs: 30_000, // Timeout for DOM to settle in milliseconds

    // LLM configuration
    modelName: "gpt-4o", // Name of the model to use
    modelClientOptions: {
      apiKey: process.env.OPENAI_API_KEY,
    }, // Configuration options for the model client

    // Browser configuration
    env: "LOCAL", // Environment to run in: LOCAL or BROWSERBASE
    localBrowserLaunchOptions: {
      cdpUrl: `http://localhost:${cdpPort}`,
      viewport: {
        width: 1024,
        height: 768,
      },
    }, // Configuration options for the local browser
  });
  await stagehand.init();

  console.log("Stagehand initialized successfully!");
})();


In [ ]:
let taskInformation = {
    employerType: 'Commercial',
    hasFEIN: 'Yes',
    idType: 'SSN',
    californiaRegistry: 'Yes',
    organizationType: "Limited Liability Company",
    idNumber: "123-45-6789",
    firstName: "John",
    lastName: "Doe",
    dateOfBirth: "1980-01-01",
    dateOfOwnership: "5/15/2020",
    industryActivity: "Other",
    industryDescription: "Software Development",
    organizationLegalName: "Doe Technologies LLC",
    stateOfIncorporation:"TX",
    fileNumber: "987654321",
    FEIN: "12-9443381",
    businessAddress: {
      country: "USA",
      street: "123 Tech Lane" + " " + "Suite 100",
      city: 'San Francisco',
      state: 'CA',
      zipcode: '94114'
    },
    mailingAddress: { // Using Lettuce Address for our customers.
      country: 'USA',
      street: '2261 Market Street STE 22617',
      city: 'San Francisco',
      state: 'CA',
      zipcode: '94114'
    },
    businessPhoneNumber: "555-123-4567",
    businessEmail: "john.doe@example.com"
  }

In [ ]:
(async () => {
    await stagehand.page.goto('https://apps.twc.texas.gov/UITAXREG/security/selfRegister.do')
})();

In [ ]:
(async () => {
  let pInfo_modal = await stagehand.page.observe('Find the first name, middle initial, last name and email address text boxes for input')
  console.log(pInfo_modal);

  let fName = pInfo_modal[0];
  //let mName = pInfo_modal[1];
  let lName = pInfo_modal[2];

  let eMail = pInfo_modal[3];

  fName.arguments = ["John"];
  //mName.arguments = ["Michael"];
  lName.arguments = [taskInformation.lastName];

  eMail.arguments = ['skunkworksauto0+TX5@gmail.com'];

  await stagehand.page.act(fName);
  //await stagehand.page.act(mName);
  await stagehand.page.act(lName);
  await stagehand.page.act(eMail);

  //exits after last instructions, why??
  let lInfo_modal = await stagehand.page.observe('Find the User ID, Password and Retype Password text boxes for input')
  console.log(lInfo_modal);

  let ID = lInfo_modal[0];
  let PW = lInfo_modal[1];
  let PW2 = lInfo_modal[2];

  ID.arguments = ['TestingIDs01'];
  PW.arguments = ['F0rm@tions'];
  PW2.arguments = ['F0rm@tions'];
  
  await stagehand.page.act(ID);
  await stagehand.page.act(PW);
  await stagehand.page.act(PW2);

  // Observe the comboboxes
  let questionSelectorBoxes = await stagehand.page.observe("Find the 'First Security Question combobox, 'Second Security Question' combobox, and 'Third Security Question' combobox");
  console.log("Observed comboboxes:", questionSelectorBoxes);

  // Extract the last option text from each combobox
  const { Security1 } = await stagehand.page.extract({
    instruction: "Get the text of the last selectable option from the 'First Security Question' combobox.",
    schema: z.object({ Security1: z.string() })
  });

  const { Security2 } = await stagehand.page.extract({
    instruction: "Get the text of the last selectable option from the 'Second Security Question' combobox.",
    schema: z.object({ Security2: z.string() })
  });

  const { Security3 } = await stagehand.page.extract({
    instruction: "Get the text of the last selectable option from the 'Third Security Question' combobox.",
    schema: z.object({ Security3: z.string() })
  });

  console.log("Last options extracted:");
  console.log("First:", Security1);
  console.log("Second:", Security2);
  console.log("Third:", Security3);

  // Define actions to select the extracted last options
  let sQuestion1 = {
    description: "The 'First Security Question' combobox, which allows the user to select different questions.",
    method: 'pressSequentially',
    arguments: [Security1, 'Enter'],
    selector: questionSelectorBoxes[0]['selector']
  };

  let sQuestion2 = {
    description: "The 'Second Security Question' combobox, which allows the user to select different questions.",
    method: 'pressSequentially',
    arguments: [Security2, 'Enter'],
    selector: questionSelectorBoxes[1]['selector']
  };

  let sQuestion3 = {
    description: "The 'Third Security Question' combobox, which allows the user to select different questions.",
    method: 'pressSequentially',
    arguments: [Security3, 'Enter'],
    selector: questionSelectorBoxes[2]['selector']
  };

  // Execute the actions
  await stagehand.page.act(sQuestion1);
  await stagehand.page.act(sQuestion2);
  await stagehand.page.act(sQuestion3);

  function getLastWord(text) {
    return text
      .replace(/\?$/, '')        // Remove trailing question mark if present
      .trim()                    // Remove surrounding whitespace
      .split(/\s+/)              // Split by whitespace
      .pop();                    // Get the last word
  }
  
  let lastWord1 = getLastWord(Security1);
  let lastWord2 = getLastWord(Security2);
  let lastWord3 = getLastWord(Security3);

  let sAnswers = await stagehand.page.observe('Find the First Security Answer, First Retype Answer, Second Security Answer, Second Retype Answer, Third Security Answer, and Third Retype Answer')

  let securityFirst = sAnswers[0];
  let retypeFirst = sAnswers[1];
  let securitySecond = sAnswers[2];
  let retypeSecond = sAnswers[3];
  let securityThird = sAnswers[4];
  let retypeThird = sAnswers[5];

  securityFirst.arguments = [lastWord1];
  retypeFirst.arguments = [lastWord1];
  securitySecond.arguments = [lastWord2];
  retypeSecond.arguments = [lastWord2];
  securityThird.arguments = [lastWord3];
  retypeThird.arguments = [lastWord3];
  
  await stagehand.page.act(securityFirst);
  await stagehand.page.act(retypeFirst);
  await stagehand.page.act(securitySecond);
  await stagehand.page.act(retypeSecond);
  await stagehand.page.act(securityThird);
  await stagehand.page.act(retypeThird);

})();

In [ ]:
(async () => {
    await stagehand.page.act("Click on the 'Next' button");
})();

In [ ]:
(async () => {
    
  //Preparing the phone number since it is split into 3
  let [areaCode, centralOfficeCode, lineNumber] = taskInformation.businessPhoneNumber.split("-");
  console.log(areaCode);
  console.log(centralOfficeCode);
  console.log(lineNumber);


  //Observing textboxes and drop-down
  let mInfo_modal = await stagehand.page.observe('Find the mailing address, city, and Zip Code text boxes for input')

  let pNumber_modal = await stagehand.page.observe('There are 4 textboxes for the "Phone Number" field, find them')
  
  let mInfo_combobox = await stagehand.page.observe("Find the 'State' combobox")

  //Assigning arguments

  //Textboxes
  let mAddress = mInfo_modal[0];
  let city = mInfo_modal[1];
  let zip = mInfo_modal[2];

  mAddress.arguments = [taskInformation.mailingAddress.street];
  city.arguments = [taskInformation.mailingAddress.city];
  zip.arguments = [taskInformation.mailingAddress.zipcode];

  //Phone Number
  pNumber_modal[0].arguments = [areaCode];
  pNumber_modal[1].arguments = [centralOfficeCode];
  pNumber_modal[2].arguments = [lineNumber];

  //Drop-down
  let dState = {
    description: "The 'State' combobox, which allows the user to select different states.",
    method: 'pressSequentially',
    arguments: [taskInformation.mailingAddress.state, 'Enter'],
    selector: mInfo_combobox[0]['selector']
  };


  //Acting

  //Textboxes
  await stagehand.page.act(mAddress);
  await stagehand.page.act(city);
  await stagehand.page.act(zip);

  //Phone Number
  await stagehand.page.act(pNumber_modal[0]);
  await stagehand.page.act(pNumber_modal[1]);
  await stagehand.page.act(pNumber_modal[2]);

  //Drop-down
  await stagehand.page.act(dState);


})();

In [ ]:
(async () => {
    await stagehand.page.act("Click on the 'Next' button");
})();

In [ ]:
(async () => {
    await stagehand.page.act("Click on the 'I Accept' button");
})();

In [ ]:
(async () => {

  const months = {
    1: "January",
    2: "February",
    3: "March",
    4: "April",
    5: "May",
    6: "June",
    7: "July",
    8: "August",
    9: "September",
    10: "October",
    11: "November",
    12: "December"
  };  
    
    //Preparing the EIN since it is split into 2
    let [Suffix, Serial] = taskInformation.FEIN.split("-");

    //Preparing the dates since they are split into 3
    let [m, d, y] = taskInformation.dateOfOwnership.split("/");

    console.log(Suffix);
    console.log(Serial);
  
    console.log(d);
    console.log(months[m]);
    console.log(y);

    //Observing textboxes and drop-down
    let eName = await stagehand.page.observe('Find the employer name text box for input')

    let FEINBoxes = await stagehand.page.observe('There are 2 text boxes on the "F.E.I.N." field, find them')
  
    let dDowns = await stagehand.page.observe("Find the 'Registrants Relationship to Employer' and 'Business Type' comboboxes")

    let yearBoxes = await stagehand.page.observe('There are 2 "Year" text boxes, find them')
    
    let monthDDowns = await stagehand.page.observe("There are 2 'Month' comboboxes, find them")

    let dayDDowns = await stagehand.page.observe("There are 2 'day' comboboxes, find them")
  
    //Assigning arguments
    //Textboxes
    let employerName = eName[0];

    let FEINPrefix = FEINBoxes[0];
    let FEINSerial = FEINBoxes[1];

    let employmentYear = yearBoxes[0];
    let wageYear = yearBoxes[1];
  
    employerName.arguments = [taskInformation.organizationLegalName];

    FEINPrefix.arguments = [Suffix];
    FEINSerial.arguments = [Serial];

    employmentYear.arguments = ["2025"];
    wageYear.arguments = ["2025"];


    //drop-downs
    let employmentMonth = {
      description: "The 'Registrant's Relationship to Employer' combobox, which allows the user to select different options.",
      method: 'pressSequentially',
      arguments: [months[m], 'Enter'],
      selector: monthDDowns[0]['selector']
    };
    
    let wageMonth = {
      description: "The 'Registrant's Relationship to Employer' combobox, which allows the user to select different options.",
      method: 'pressSequentially',
      arguments: [months[m], 'Enter'],
      selector: monthDDowns[1]['selector']
    };

    let employmentDay = {
      description: "The 'Registrant's Relationship to Employer' combobox, which allows the user to select different options.",
      method: 'pressSequentially',
      arguments: [d, 'Enter'],
      selector: dayDDowns[0]['selector']
    };
    
    let wageDay = {
      description: "The 'Registrant's Relationship to Employer' combobox, which allows the user to select different options.",
      method: 'pressSequentially',
      arguments: [d, 'Enter'],
      selector: dayDDowns[1]['selector']
    };
    

    let employerRelationship = {
      description: "The 'Registrant's Relationship to Employer' combobox, which allows the user to select different options.",
      method: 'pressSequentially',
      arguments: ['Owner', 'Enter'],
      selector: dDowns[0]['selector']
    };
    
    let businessType = {
      description: "The 'Business Type' combobox, which allows the user to select different buesiness types.",
      method: 'pressSequentially',
      arguments: ['Limited Liability Company', 'Enter'],
      selector: dDowns[1]['selector']
    };
  
    //Acting
  
    //Textboxes
    await stagehand.page.act(employerName);
    
    await stagehand.page.act(FEINPrefix);
    await stagehand.page.act(FEINSerial);

    await stagehand.page.act(employmentYear);
    await stagehand.page.act(wageYear);
  
    //Drop-downs
    await stagehand.page.act(employmentMonth);
    await stagehand.page.act(wageMonth);

    await stagehand.page.act(employmentDay);
    await stagehand.page.act(wageDay);
  
    await stagehand.page.act(employerRelationship);
    await stagehand.page.act(businessType);

    //Checkbox
    await stagehand.page.act("Click on the 'Regular Employment' checkbox");
  
  })();

In [ ]:
(async () => {
    await stagehand.page.act("Click on the 'Next' button");
})();

In [ ]:
(async () => {


  //Observing textboxes and drop-down
  let corpName = await stagehand.page.observe('Find the Corporation Name text box for input')

  let Quarter = await stagehand.page.observe("Find the 'First $1,500 Quarter' combobox and textbox")

  let yearsLiable = await stagehand.page.observe('There are 4 "Years Liable for Federal Unemployment Tax" text boxes, find them')
  
  let Employee = await stagehand.page.observe("Find the 'Current No. of Texas Employees' textbox")

  //Assigning arguments
  //Textboxes
  let corporationName = corpName[0];

  let quarterYear = Quarter[1];

  let liableYear1 = yearsLiable[0];

  let employeesNumber = Employee[0];

  corporationName.arguments = [taskInformation.organizationLegalName];

  quarterYear.arguments = ["2025"]; 

  liableYear1.arguments = ["2025"];
  employeesNumber.arguments = ["1"];


  //drop-downs
  let quarterMonths = {
    description: "The 'First $1,500 Quarter' combobox, which allows the user to select different options.",
    method: 'pressSequentially',
    arguments: ["Jul", 'Enter'],
    selector: Quarter[0]['selector']
  }; 

  //Acting
  await stagehand.page.act(corporationName);
  
  await stagehand.page.act(quarterMonths);
  await stagehand.page.act(quarterYear);

  await stagehand.page.act(liableYear1);
  await stagehand.page.act(employeesNumber);

  //Checkbox
  await stagehand.page.act("Click on the 'No' checkbox");

})();

In [ ]:
(async () => {
    await stagehand.page.act("Click on the 'Next' button");
})();

In [ ]:
(async () => {
    await stagehand.page.act("Click on the 'No' checkbox");
})();

In [ ]:
(async () => {
    await stagehand.page.act("Click on the 'Next' button");
})();

In [ ]:
(async () => {

  const months = {
    1: "January",
    2: "February",
    3: "March",
    4: "April",
    5: "May",
    6: "June",
    7: "July",
    8: "August",
    9: "September",
    10: "October",
    11: "November",
    12: "December"
  };  

  //Preparing the dates since they are split into 3
  let [m, d, y] = taskInformation.dateOfOwnership.split("/");
  
    //Observing textboxes and drop-down
    let fileNum = await stagehand.page.observe('Find the Filing Number text box for input')

    let fileDate = await stagehand.page.observe("On the Filing Date field, find the 'Month' combobox, the 'Day' Combobox, and the 'Year' textbox")

    let agentDetails = await stagehand.page.observe("Under 'Registered Agent Information', find the 'Name', 'Mailing address', 'City', and 'Zip code' textboxes")
    
    let State = await stagehand.page.observe("Find the 'State' combobox")
  
    //Assigning arguments
    //Textboxes
    let filingNumber = fileNum[0];

    let filingYear = fileDate[2];

    let agentName = agentDetails[0];
    let agentAddy = agentDetails[1];
    let agentCity = agentDetails[2];
    let agentZip = agentDetails[3];
  
    filingNumber.arguments = [taskInformation.fileNumber];

    filingYear.arguments = [y]; 

    agentName.arguments = [taskInformation.firstName + " " + taskInformation.lastName];
    agentAddy.arguments = [taskInformation.mailingAddress.street];
    agentCity.arguments = [taskInformation.mailingAddress.city];
    agentZip.arguments = [taskInformation.mailingAddress.zipcode];


    //drop-downs
    let filingMonth = {
      description: "On the 'Filing Date' field, the 'Month' combobox, which allows the user to select different months.",
      method: 'pressSequentially',
      arguments: [months[m], 'Enter'],
      selector: fileDate[0]['selector']
    }; 

    let filingDay = {
      description: "On the 'Filing Date' field, the 'Day' combobox, which allows the user to select different numbers.",
      method: 'pressSequentially',
      arguments: [d, 'Enter'],
      selector: fileDate[1]['selector']
    }; 

    let agentState = {
      description: "The 'State' combobox, which allows the user to select different states.",
      method: 'pressSequentially',
      arguments: [taskInformation.mailingAddress.state, 'Enter'],
      selector: State[0]['selector']
    }; 
  
    //Acting
    await stagehand.page.act(filingNumber);
    
    await stagehand.page.act(filingMonth);
    await stagehand.page.act(filingDay);
    await stagehand.page.act(filingYear);

    await stagehand.page.act(agentName);
    await stagehand.page.act(agentAddy);
    await stagehand.page.act(agentCity);
    await stagehand.page.act(agentState);
    await stagehand.page.act(agentZip);
  
  })();

In [ ]:
(async () => {
    await stagehand.page.act("Click on the 'Next' button");
})();

In [ ]:
(async () => {
  
    //Preparing the dates since they are split into 3
    let [area, group, serial] = taskInformation.idNumber.split("-");
    
    //Observing textboxes and drop-down
    let officerBoxes = await stagehand.page.observe('Find the "Name", "Middle initial", "Last name", "Email Address", "Residence Address", "City", and "Zip code"  textboxes for input')

    let SSNBoxes = await stagehand.page.observe("There are 3 textboxes on the 'social security number' field, find them")

    let dDowns = await stagehand.page.observe("Find the Title and State comboboxes.")
  
    //Assigning arguments
    //Textboxes
    let officerGiven = officerBoxes[0];
    let officerLast = officerBoxes[2];
    
    let officerMail = officerBoxes[3];

    let officerAddy = officerBoxes[4];
    let officerCity = officerBoxes[5];
    let officerZip = officerBoxes[6];

    let areaID = SSNBoxes[0];
    let groupID = SSNBoxes[1];
    let serialID = SSNBoxes[2];
  
    officerGiven.arguments = [taskInformation.firstName];
    officerLast.arguments = [taskInformation.lastName]; 

    officerMail.arguments = [taskInformation.businessEmail]; 

    officerAddy.arguments = [taskInformation.businessAddress.street];
    officerCity.arguments = [taskInformation.businessAddress.city];
    officerZip.arguments = [taskInformation.businessAddress.zipcode];
    
    areaID.arguments = [area];
    groupID.arguments = [group];
    serialID.arguments = [serial];

    //drop-downs
    let officerTitle = {
      description: "The 'Title' combobox, which allows the user to select different options.",
      method: 'pressSequentially',
      arguments: ["Corporate Officer", 'Enter'],
      selector: dDowns[0]['selector']
    }; 

    let officerState = {
      description: "The 'State' combobox, which allows the user to select different states.",
      method: 'pressSequentially',
      arguments: [taskInformation.businessAddress.state, 'Enter'],
      selector: dDowns[1]['selector']
    }; 
  
    //Acting
    await stagehand.page.act(officerGiven);
    await stagehand.page.act(officerLast);

    await stagehand.page.act(areaID);
    await stagehand.page.act(groupID);
    await stagehand.page.act(serialID);

    await stagehand.page.act(officerTitle);

    await stagehand.page.act(officerMail);

    await stagehand.page.act(officerAddy);
    await stagehand.page.act(officerCity);
    await stagehand.page.act(officerState);
    await stagehand.page.act(officerZip);
  
  })();

In [ ]:
(async () => {
    await stagehand.page.act("Click on the 'Next' button");
})();

In [ ]:
(async () => {
    await stagehand.page.act("Click on the 'Next' button");
})();

In [ ]:
(async () => {
  
    //Preparing the dates since they are split into 3
    let [area, office, serial] = taskInformation.businessPhoneNumber.split("-");
    
    //Observing textboxes and drop-down
    let addyBoxes = await stagehand.page.observe('Find the "Mailing Address" textbox, "City" textbox, "State" combobox, "Zip code" textbox fields')

    let pNumber = await stagehand.page.observe("There are 3 textboxes on the 'Phone Number' field, find them")
  
    //Assigning arguments
    //Textboxes
    let employerStreet = addyBoxes[0];
    let employerCity = addyBoxes[1];
    let employerZip = addyBoxes[3];
    
    let areaCode = pNumber[0];
    let centralOffice = pNumber[1];
    let serialNum = pNumber[2];
  
    employerStreet.arguments = [taskInformation.mailingAddress.street];
    employerCity.arguments = [taskInformation.mailingAddress.city]; 
    employerZip.arguments = [taskInformation.mailingAddress.zipcode]; 

    areaCode.arguments = [area];
    centralOffice.arguments = [office];
    serialNum.arguments = [serial];

    //drop-downs
    let employerState = {
      description: "The 'State' combobox, which allows the user to select different states.",
      method: 'pressSequentially',
      arguments: [taskInformation.mailingAddress.state, 'Enter'],
      selector: addyBoxes[2]['selector']
    }; 
  
    //Acting
    await stagehand.page.act(employerStreet);
    await stagehand.page.act(employerCity);
    await stagehand.page.act(employerState);
    await stagehand.page.act(employerZip);

    await stagehand.page.act(areaCode);
    await stagehand.page.act(centralOffice);
    await stagehand.page.act(serialNum);

    await stagehand.page.act("Click on the 'Yes' checkbox");
  
  })();

In [ ]:
(async () => {
    await stagehand.page.act("Click on the 'Next' button");
})();

In [ ]:
(async () => {
  
  //Observing textboxes and drop-down
  let locationBoxes = await stagehand.page.observe('Find the "Trade Name", "Physical Address", "City", and "Zip code" textboxes for input')

  //Assigning arguments
  //Textboxes
  let tradeName = locationBoxes[0];

  let tradeAddy = locationBoxes[1];
  let tradeCity = locationBoxes[2];
  let tradeZip = locationBoxes[3];

  tradeName.arguments = [taskInformation.organizationLegalName];

  tradeAddy.arguments = [taskInformation.businessAddress.street]; 
  tradeCity.arguments = [taskInformation.businessAddress.city]; 
  tradeZip.arguments = [taskInformation.businessAddress.zipcode];

  //Acting
  await stagehand.page.act(tradeName);
  
  await stagehand.page.act(tradeAddy);
  await stagehand.page.act(tradeCity);
  await stagehand.page.act(tradeZip);
  })();

In [ ]:
(async () => {
    await stagehand.page.act("Click on the 'Next' button");
})();

In [ ]:
(async () => {
    await stagehand.page.act("Click on the 'Next' button");
})();

In [ ]:
(async () => {
    let txt = await stagehand.page.observe("Find the 'Nature of Activity' textbox");
    
    txt[0].arguments = [taskInformation.industryDescription];

    await stagehand.page.act(txt[0])
})();

In [ ]:
(async () => {
    await stagehand.page.act("Click on the 'Next' button");
})();